# Ragas Evaluation

In [19]:
# LIBRARIES IMPORT
import pandas as pd
import numpy as np

import ragas
import openai
import langchain
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

In [9]:
# MODULES IMPORT
from query_chatbot_by_retriever import load_database
from get_chunk_stats import reconstruct_chunks

## Import the data

In [17]:
# Load Chroma database
vector_db_path = "../chroma_db"
vector_db = load_database(vector_db_path)[0]

# Get the chunks
chunks = reconstruct_chunks(vector_db)

2024-05-21 17:45:05.218 | INFO     | query_chatbot_by_retriever:load_database:179 - Loading the vector database.
2024-05-21 17:45:05.267 | INFO     | query_chatbot_by_retriever:load_database:185 - Chunks in the database: 1312
2024-05-21 17:45:05.268 | SUCCESS  | query_chatbot_by_retriever:load_database:187 - Vector database prepared successfully.

2024-05-21 17:45:05.269 | INFO     | get_chunk_stats:reconstruct_chunks:224 - Reconstructing the chunks from the vector database...
2024-05-21 17:45:05.347 | SUCCESS  | get_chunk_stats:reconstruct_chunks:238 - Reconstructed 1312 chunks from the vector database successfully.



## Create the test set

In [20]:
generator = TestsetGenerator.with_openai()
testset = generator.generate_with_langchain_docs(chunks, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})
testset.to_pandas()


/tmp/ipykernel_135824/966000715.py:1: DeprecationWarning: The function with_openai was deprecated in 0.1.4, and will be removed in the 0.2.0 release. Use from_langchain instead.
  generator = TestsetGenerator.with_openai()


embedding nodes:   0%|          | 0/2624 [00:00<?, ?it/s]

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/tess01hp/anaconda3/envs/biopyassistantenv/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/tess01hp/anaconda3/envs/biopyassistantenv/lib/python3.11/site-packages/ragas/executor.py", line 96, in run
    results = self.loop.run_until_complete(self._aresults())
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tess01hp/anaconda3/envs/biopyassistantenv/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/home/tess01hp/anaconda3/envs/biopyassistantenv/lib/python3.11/site-packages/ragas/executor.py", line 84, in _aresults
    raise e
  File "/home/tess01hp/anaconda3/envs/biopyassistantenv/lib/python3.11/site-packages/ragas/executor.py", line 79, in _aresults
    r = await future
        ^^^^^^^^^^^^
  File "/home/tess01hp/anaconda3/envs/biopyassistantenv/lib/python3.11

ExceptionInRunner: The runner thread which was running the jobs raised an exeception. Read the traceback above to debug it. You can also pass `raise_exceptions=False` incase you want to show only a warning message instead.

In [ ]:
questions = testset.to_pandas()["question"].to_list()
ground_truth = testset.to_pandas()["ground_truth"].to_list()

In [ ]:
from datasets import Dataset

questions = testset.to_pandas()["question"].to_list()
ground_truth = testset.to_pandas()["ground_truth"].to_list()

data = {"question": [], "answer": [], "contexts": [], "ground_truth": ground_truth}

for query in questions:
    data["question"].append(query)
    data["answer"].append(rag_chain.invoke(query))
    data["contexts"].append([doc.page_content for doc in retriever.get_relevant_documents(query)])

dataset = Dataset.from_dict(data)